# Pfizer 2023 Hail Workshop
## 01: Common Variant Analysis

Learning objectives:
1) Use simple python code and Jupyter notebooks.

2) Perform simple quality control with Hail.

3) Associate phenotype to common variation with Hail

# We've come along way since 2014!

Part of what we think is so exciting about Hail is that it has coincided with a larger shift in the data science community.

Eight years ago, most computational biologists at Broad analyzed genetic data using command-line tools, and took advantage of research compute clusters by explicitly using scheduling frameworks like LSF or Sun Grid Engine.

Three big things have changed since then. First, since Hail is just a Python library, it is easy to integrate genetic analyses in Hail with other kinds of analysis using pandas and numpy. Second, Jupyter Notebooks facilitate interactive and visual interaction with analysis results. Finally, every major public cloud supports Hail clusters which enable analysts to scale their analysis from eight laptop cores to ten thousand cloud cores.

# Using Jupyter
### Running cells
Evaluate cells using SHIFT + ENTER. Select the next cell and run it

In [ ]:
print('Hello, world')

### Modes

Jupyter has two modes, a **navigation mode** and an **editor mode**.

#### Navigation mode:

 - <font color="blue"><strong>BLUE</strong></font> cell borders
 - `UP` / `DOWN` move between cells
 - `ENTER` while a cell is selected will move to **editing mode**.
 - Many letters are keyboard shortcuts! This is a common trap.
 
#### Editor mode:

 - <font color="green"><strong>GREEN</strong></font> cell borders
 - `UP` / `DOWN`/ move within cells before moving between cells.
 - `ESC` will return to **navigation mode**.
 - `SHIFT + ENTER` will evaluate a cell and return to **navigation mode**.

### Cell types

There are several types of cells in Jupyter notebooks. The two you will see here are **Markdown** (text) and **Code**.

In [ ]:
# This is a code cell
my_variable = 5

**This is a markdown cell**, so even if something looks like code (as below), it won't get executed!

my_variable += 1

In [ ]:
my_variable

If the last line of the cell is a variable, Jupyter will try to display it. This works for most simple data types like numbers and lists.

### Tips and tricks

Keyboard shortcuts:

 - `SHIFT + ENTER` to evaluate a cell or move past a markdown cell
 - `ESC` to return to navigation mode
 - `y` to turn a markdown cell into code
 - `m` to turn a code cell into markdown
 - `a` to add a new cell **above** the currently selected cell
 - `b` to add a new cell **below** the currently selected cell
 - `d, d` (repeated) to delete the currently selected cell
 - `TAB` to activate code completion
 
To try this out, create a new cell below this one using `b`, and print `my_variable` by starting with `print(my` and pressing `TAB`!

# Set up our Python environment

If you need installation instructions, please visit our Getting Started page (https://hail.is/docs/0.2/getting_started.html)

In [1]:
import hail as hl

Let's initialize Hail and set up plotting to display inline in the notebook. Initialization is *not required*, but we do it here so you can see the information Hail prints when it initializes. You'll see some text on a red background, that means Hail started successfully!

In [2]:
hl.init()
hl.plot.output_notebook()

2023-01-11 13:25:08.052 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://148.168.60.57:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.107-2387bb00ceee
LOGGING: writing to /Users/dking/projects/pfizer-2023-01-11/01/hail-20230111-1325-0.2.107-2387bb00ceee.log


Loading BokehJS ...

The demonstration materials are designed to work on a small (~20MB) downsampled chunk of the public 1000 Genomes dataset.


It is possible to call command-line utilities from Jupyter by prefixing a line with a `!`:

In [3]:
! ls -1 resources/

1kg.mt
1kg.vcf.bgz
1kg_annotations.txt
ensembl_gene_annotations.txt
pca_scores.ht


# Quality Control

### Import data from VCF

The [Variant Call Format (VCF)](https://en.wikipedia.org/wiki/Variant_Call_Format) is a common file format for representing genetic data collected on multiple individuals (samples).

Hail's [import_vcf](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_vcf) function can read this format.

Although VCF is an easy-to-read text format for humans, text formats are not as efficient as binary formats. As such, we wil import the VCF and save it as a Hail native file format. The resulting file is **much** faster to process because it is scalable and easily parallelizable. We recommend always converting to Hail format immediately after an import of any text format like VCF and TSV.


In [4]:
vcf = hl.import_vcf('resources/1kg.vcf.bgz', min_partitions=4)
vcf.write('resources/1kg.mt', overwrite=True)

2023-01-11 13:38:15.269 Hail: INFO: scanning VCF for sortedness...
2023-01-11 13:38:21.642 Hail: INFO: Coerced sorted VCF - no additional import work to do
2023-01-11 13:38:26.862 Hail: INFO: wrote matrix table with 10879 rows and 284 columns in 4 partitions to resources/1kg.mt


### Read 1KG into Hail

We represent genetic data as a Hail [`MatrixTable`](https://hail.is/docs/0.2/overview/matrix_table.html), and name our variable `mt` to indicate this.

In [5]:
mt = hl.read_matrix_table('resources/1kg.mt')

### What is a `MatrixTable`?

Let's explore it!

You can see:
 - **numeric** types:
     - integers (`int32`, `int64`), e.g. `5`
     - floating point numbers (`float32`, `float64`), e.g. `5.5` or `3e-8`
 - **strings** (`str`), e.g. `"Foo"`
 - **boolean** values  (`bool`) e.g. `True`
 - **collections**:
     - arrays (`array`), e.g. `[1,1,2,3]`
     - sets (`set`), e.g. `{1,3}`
     - dictionaries (`dict`), e.g. `{'Foo': 5, 'Bar': 10}`
 - **genetic data types**:
     - loci (`locus`), e.g. `[GRCh37] 1:10000` or `[GRCh38] chr1:10024`
     - genotype calls (`call`), e.g. `0/2` or `1|0`

In [6]:
mt.describe(widget=True)

Unlike simple data types, like integers and strings, Matrix Tables do not have a useful printed form in Jupyter.

In [7]:
mt

### `show`

You can, however, explicitly request a useful printed form. For example, you can show individual fields like the sample ID (`s`), 

In [8]:
mt.s.show()

""
s
str
"""HG00096"""
"""HG00099"""
"""HG00105"""
"""HG00118"""
"""HG00129"""
"""HG00148"""
"""HG00177"""
"""HG00182"""


the locus (`locus`),

In [9]:
mt.locus.show()

,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"
1:2252970,"[""C"",""T""]"
1:2284195,"[""T"",""C""]"
1:2779043,"[""T"",""C""]"


the called genotype (`GT`),

In [10]:
mt.GT.show()

,,,,,
,,'HG00096','HG00099','HG00105','HG00118'
locus,alleles,GT,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call,call
1:904165,"[""G"",""A""]",0/0,0/0,0/0,0/0
1:909917,"[""G"",""A""]",0/0,0/0,0/0,0/0
1:986963,"[""C"",""T""]",0/0,0/0,0/0,0/0
1:1563691,"[""T"",""G""]",NA,0/0,0/0,0/0
1:1707740,"[""T"",""G""]",0/1,0/1,0/1,0/0
1:2252970,"[""C"",""T""]",0/0,NA,0/0,0/0
1:2284195,"[""T"",""C""]",1/1,0/1,0/1,0/1


or all the entry fields of the Matrix Table:

In [12]:
?mt.show

In [13]:
mt.show(n_rows=5, n_cols=3, include_row_fields=True)

+---------------+------------+------+----------+----------+--------------+
| locus         | alleles    | rsid |     qual | filters  | info.AC      |
+---------------+------------+------+----------+----------+--------------+
| locus<GRCh37> | array<str> | str  |  float64 | set<str> | array<int32> |
+---------------+------------+------+----------+----------+--------------+
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:909917      | ["G","A"]  | NA   | 1.58e+03 | NA       | [18]         |
| 1:986963      | ["C","T"]  | NA   | 3.98e+02 | NA       | [5]          |
| 1:1563691     | ["T","G"]  | NA   | 1.09e+03 | NA       | [64]         |
| 1:1707740     | ["T","G"]  | NA   | 9.35e+04 | NA       | [997]        |
+---------------+------------+------+----------+----------+--------------+

+----------------+---------+-------------------+----------------------+
| info.AF        | info.AN | info.BaseQRankSum | info.ClippingRankSum |
+----------------+---------+-------------------+----------------------+
| array<float64> |   int32 |           float64 |              float64 |
+----------------+---------+-------------------+----------------------+
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [3.73e-03]     |    4830 |         -1.48e+00 |             1.26e-01 |
| [1.09e-03]     |    4588 |          1.25e+00 |            -3.77e+00 |
| [1.30e-02]     |    4766 |         -3.87e+01 |            -5.39e+00 |
| [1.98e-01]     |    5034 |         -4.04e+01 |            -2.87e-01 |
+----------------+---------+-------------------+----------------------+

+---------+---------+----------+---------------------+----------------------+
| info.DP | info.DS |  info.FS | info.HaplotypeScore | info.InbreedingCoeff |
+---------+---------+----------+---------------------+----------------------+
|   int32 |    bool |  float64 |             float64 |              float64 |
+---------+---------+----------+---------------------+----------------------+
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   14671 |   False | 5.52e+00 |                  NA |            -5.00e-04 |
|   12398 |   False | 8.34e-01 |                  NA |             1.26e-02 |
|   15357 |   False | 1.90e+03 |                  NA |             2.70e-02 |
|   19902 |   False | 3.31e+00 |                  NA |             3.87e-02 |
+---------+---------+----------+---------------------+----------------------+

+--------------+----------------+----------+----------+----------------+
| info.MLEAC   | info.MLEAF     |  info.MQ | info.MQ0 | info.MQRankSum |
+--------------+----------------+----------+----------+----------------+
| array<int32> | array<float64> |  float64 |    int32 |        float64 |
+--------------+----------------+----------+----------+----------------+
| [514]        | [1.02e-01]     | 5.91e+01 |        0 |       1.45e+00 |
| [15]         | [3.11e-03]     | 5.91e+01 |        0 |       1.76e+00 |
| [3]          | [6.54e-04]     | 5.79e+01 |        0 |       5.86e-01 |
| [22]         | [4.62e-03]     | 5.90e+01 |        0 |       1.31e+00 |
| [983]        | [1.95e-01]     | 5.83e+01 |        0 |       9.48e+00 |
+--------------+----------------+----------+----------+----------------+

+----------+---------------------+----------+--------------+--------------+
|  info.QD | info.ReadPosRankSum | info.set | 'HG00096'.GT | 'HG00096'.AD |
+----------+---------------------+----------+--------------+--------------+
|  float64 |             float64 | str      | call         | array<int32> |
+----------+---------------------+----------+--------------+--------------+
| 1.50e+01 |            6.29e+00 | NA       | 0/0          | [4,0]        |
| 1.37e+01 |           -1.43e+00 | NA       | 0/0          | [4,0]        |
| 1.73e+01 |            7.10e-01 | NA       | 0/0          | [3,0]        |
| 5.05e+00 |            1.15e+00 | NA       | NA           | NA           |
| 1.36e+01 |            2.26e+00 |

In [15]:
mt = mt.annotate_cols(rand_pheno = hl.rand_norm(5, 1))
mt.cols().show()

,
s,rand_pheno
str,float64
"""HG00096""",5.44e+00
"""HG00099""",4.62e+00
"""HG00105""",3.00e+00
"""HG00118""",4.27e+00
"""HG00129""",5.33e+00
"""HG00148""",6.86e+00
"""HG00177""",5.84e+00
"""HG00182""",4.70e+00


### `summarize`
`summarize` Prints (potentially) useful information about any field or object:

In [16]:
mt.DP.summarize()

Non-missing,3045432 (98.57%)
Missing,44204 (1.43%)
Minimum,0
Maximum,68
Mean,7.16
Std Dev,4.12


In [17]:
mt.AD.summarize()

Non-missing,3045432 (98.57%)
Missing,44204 (1.43%)
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,6090864 (100.00%)
Missing,0
Minimum,0
Maximum,68
Mean,3.58
Std Dev,4.13


You can even summarize the result of calling a Hail function on a fields!

In [18]:
hl.len(mt.alleles).summarize()

Non-missing,10879 (100.00%)
Missing,0
Minimum,2
Maximum,2
Mean,2.00
Std Dev,0.00


### `count`

`MatrixTable.count` returns a tuple with the number of rows (variants) and number of columns (samples).

In [19]:
mt.count()

(10879, 284)

### Hail has functions built for genetics

One of these is `hl.summarize_variants` which prints useful statistics about the loci and alleles present in this dataset. This is different from `FIELD.summarize()` which is not genetics-aware.

In [20]:
hl.summarize_variants(mt)

Number of alleles,Count
2,10879
Allele type,Count
SNP,10879
Metric,Value
Transitions,8578
Transversions,2301
Ratio,3.73
Contig,Count
1,909
2,844


# Annotation and quality control

## Integrate sample information

This is a text file containing phenotype information:

In [21]:
! head resources/1kg_annotations.txt

Sample	Population	SuperPopulation	isFemale	PurpleHair	CaffeineConsumption	HasSixToes
HG00096	GBR	EUR	false	false	4	false
HG00097	GBR	EUR	true	true	4	false
HG00098	GBR	EUR	false	false	5	false
HG00099	GBR	EUR	true	false	4	false
HG00100	GBR	EUR	true	false	5	false
HG00101	GBR	EUR	false	true	1	false
HG00102	GBR	EUR	true	true	6	false
HG00103	GBR	EUR	false	true	5	false
HG00104	GBR	EUR	true	false	5	false


We can import in (and, indeed, any TSV) as a [Hail Table](https://hail.is/docs/0.2/overview/table.html) with [hl.import_table](https://hail.is/docs/0.2/methods/impex.html?highlight=import_table#hail.methods.import_table).

We call it "sa" for "sample annotations".

In [22]:
sa = hl.import_table('resources/1kg_annotations.txt', 
                      impute=True, 
                      key='Sample')

2023-01-11 13:46:01.920 Hail: INFO: wrote table with 3501 rows in 1 partition to /tmp/persist_tablea7sMpyRVyk
2023-01-11 13:46:02.937 Hail: INFO: Reading table to impute column types
2023-01-11 13:46:04.672 Hail: INFO: Finished type imputation
  Loading field 'Sample' as type str (imputed)
  Loading field 'Population' as type str (imputed)
  Loading field 'SuperPopulation' as type str (imputed)
  Loading field 'isFemale' as type bool (imputed)
  Loading field 'PurpleHair' as type bool (imputed)
  Loading field 'CaffeineConsumption' as type int32 (imputed)
  Loading field 'HasSixToes' as type bool (imputed)


While we can see the names and types of fields in the logging messages, we can also `show` this table:

In [23]:
sa.show()

,,,,,,
Sample,Population,SuperPopulation,isFemale,PurpleHair,CaffeineConsumption,HasSixToes
str,str,str,bool,bool,int32,bool
"""HG00096""","""GBR""","""EUR""",False,False,4,False
"""HG00097""","""GBR""","""EUR""",True,True,4,False
"""HG00098""","""GBR""","""EUR""",False,False,5,False
"""HG00099""","""GBR""","""EUR""",True,False,4,False
"""HG00100""","""GBR""","""EUR""",True,False,5,False
"""HG00101""","""GBR""","""EUR""",False,True,1,False
"""HG00102""","""GBR""","""EUR""",True,True,6,False
"""HG00103""","""GBR""","""EUR""",False,True,5,False


And we can `summarize` each field in `sa`:

In [24]:
sa.summarize()

2023-01-11 13:46:37.800 Hail: INFO: Coerced sorted dataset


Non-missing,3500 (100.00%)
Missing,0
Min Size,7
Max Size,7
Mean Size,7.00
Sample Values,"['HG00096', 'HG00097', 'HG00098', 'HG00099', 'HG00100']"
Non-missing,3500 (100.00%)
Missing,0
Min Size,3
Max Size,3
Mean Size,3.00


## Annotate our `MatrixTable` callset with the sample annotations

It just takes one line:

In [34]:
sa.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'Sample': str 
    'Population': str 
    'SuperPopulation': str 
    'isFemale': bool 
    'PurpleHair': bool 
    'CaffeineConsumption': int32 
    'HasSixToes': bool 
----------------------------------------
Key: ['Sample']
----------------------------------------


In [35]:
mt = mt.annotate_cols(pheno = sa.index(mt.s, all_matches=True))
mt.pheno.show()

2023-01-11 13:56:48.464 Hail: INFO: Coerced sorted dataset


,
s,pheno
str,"array<struct{Population: str, SuperPopulation: str, isFemale: bool, PurpleHair: bool, CaffeineConsumption: int32, HasSixToes: bool}>"
"""HG00096""","[(""GBR"",""EUR"",False,False,4,False)]"
"""HG00099""","[(""GBR"",""EUR"",True,False,4,False)]"
"""HG00105""","[(""GBR"",""EUR"",False,False,4,False)]"
"""HG00118""","[(""GBR"",""EUR"",True,False,3,False)]"
"""HG00129""","[(""GBR"",""EUR"",False,False,6,False)]"
"""HG00148""","[(""GBR"",""EUR"",False,True,2,False)]"
"""HG00177""","[(""FIN"",""EUR"",True,True,4,False)]"
"""HG00182""","[(""FIN"",""EUR"",False,False,2,False)]"


In [36]:
mt = mt.annotate_cols(pheno = sa[mt.s])

### What's going on here?

Understanding what's going on here is a bit more difficult. To understand, we need to understand a few pieces:

#### 1. `annotate` methods

In Hail, `annotate` methods refer to **adding new fields**. 

 - `MatrixTable`'s `annotate_cols` adds new column (**sample**) fields.
 - `MatrixTable`'s `annotate_rows` adds new row (**variant**) fields.
 - `MatrixTable`'s `annotate_entries` adds new entry (**genotype**) fields.
 - `Table`'s `annotate` adds new row fields.

In the above cell, we are adding a new column (**sample**) field called "pheno". This field should be the values in our table `sa` associated with the sample ID `s` in our `MatrixTable` - that is, this is performing a **join**.

Python uses square brackets to look up values in dictionaries:

    d = {'foo': 5, 'bar': 10}
    d['foo']

You should think of this in much the same way - for each column of `mt`, we are looking up the fields in `sa` using the sample ID `s`.

Let's look at where does this go into the `MatrixTable`

In [26]:
mt.describe(widget=True)

In [27]:
mt.cols().show()

2023-01-11 13:48:08.069 Hail: INFO: Coerced sorted dataset


+-----------+------------+------------------+-----------------------+
| s         | rand_pheno | pheno.Population | pheno.SuperPopulation |
+-----------+------------+------------------+-----------------------+
| str       |    float64 | str              | str                   |
+-----------+------------+------------------+-----------------------+
| "HG00096" |   5.44e+00 | "GBR"            | "EUR"                 |
| "HG00099" |   4.62e+00 | "GBR"            | "EUR"                 |
| "HG00105" |   3.00e+00 | "GBR"            | "EUR"                 |
| "HG00118" |   4.27e+00 | "GBR"            | "EUR"                 |
| "HG00129" |   5.33e+00 | "GBR"            | "EUR"                 |
| "HG00148" |   6.86e+00 | "GBR"            | "EUR"                 |
| "HG00177" |   5.84e+00 | "FIN"            | "EUR"                 |
| "HG00182" |   4.70e+00 | "FIN"            | "EUR"                 |
| "HG00242" |   3.81e+00 | "GBR"            | "EUR"                 |
| "HG00254" |   4.70e+00 | "GBR"            | "EUR"                 |
+-----------+------------+------------------+-----------------------+

+----------------+------------------+---------------------------+
| pheno.isFemale | pheno.PurpleHair | pheno.CaffeineConsumption |
+----------------+------------------+---------------------------+
|           bool |             bool |                     int32 |
+----------------+------------------+---------------------------+
|          False |            False |                         4 |
|           True |            False |                         4 |
|          False |            False |                         4 |
|           True |            False |                         3 |
|          False |            False |                         6 |
|          False |             True |                         2 |
|           True |             True |                         4 |
|          False |            False |                         2 |
|          False |            False |                         1 |
|           True |            False |                         2 |
+----------------+------------------+---------------------------+

+------------------+
| pheno.HasSixToes |
+------------------+
|             bool |
+------------------+
|            False |
|            False |
|            False |
|            False |
|            False |
|            False |
|            False |
|            False |
|            False |
|            False |
+------------------+
showing top 10 rows

## Query the phenotype fields

What’s the fraction of samples with `PurpleHair`?

In [28]:
mt.aggregate_cols(hl.agg.fraction(mt.pheno.PurpleHair))

0.49295774647887325

How many people are in each self-reported major ancestry group?

In [29]:
mt.aggregate_cols(hl.agg.counter(mt.pheno.SuperPopulation))

{'AFR': 76, 'AMR': 34, 'EAS': 72, 'EUR': 47, 'SAS': 55}

## Sample QC

We'll start with examples of sample QC.

Hail has the function [hl.sample_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc) to compute a list of useful statistics about samples from sequencing data. This function adds a new column field, `sample_qc`, with the computed statistics.

**Click the link** above to see the documentation, which lists the fields and their descriptions.

In [30]:
mt = hl.sample_qc(mt)

In [31]:
mt.describe(widget=True)

Hail includes a plotting library built on [bokeh](https://bokeh.pydata.org/en/latest/index.html) that makes it easy to visualize fields of Hail tables and matrix tables.

Let's visualize the distribution of `Mean DP` (`DP` = Read Depth) to `Call Rate`:

In [33]:
p = hl.plot.scatter(x=mt.sample_qc.dp_stats.mean,
                    y=mt.sample_qc.call_rate,
                    xlabel='Mean DP',
                    ylabel='Call Rate',
                    hover_fields={'ID': mt.s},
                    size=3,
                   width=400,
                   height=400)
hl.plot.show(p)

### Filter columns using generated QC statistics

Before filtering samples, we should compute a raw sample count:

In [37]:
mt.count_cols()

284

`filter_cols` removes entire columns from the matrix table. Here, we keep columns (samples) where the `call_rate` is over 95%:

In [38]:
mt = mt.filter_cols(mt.sample_qc.call_rate >= 0.95)

We can compute a final sample count:

In [39]:
mt.count_cols()

267

Let's look at the distribution of depths.

In [40]:
p = hl.plot.histogram(mt.DP, legend='DP', title='DP Histogram')
hl.plot.show(p)

You can also create plots yourself using your favorite plotting library. Under the covers, the `hl.plot` module just collects that data using standard Hail aggregators.

In [41]:
dp_hist = mt.aggregate_entries(hl.agg.hist(mt.DP, 0, 30, 30))

In [42]:
dp_hist

Struct(bin_edges=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0], bin_freq=[144, 83391, 156537, 229447, 285115, 312632, 312301, 292769, 256012, 216491, 176019, 139873, 107217, 81077, 60755, 44180, 32717, 23918, 17528, 12654, 9324, 6849, 4934, 3741, 2716, 1958, 1502, 1037, 758, 861], n_smaller=0, n_larger=801)

In [43]:
import bokeh
from bokeh.plotting import figure, show

p = figure(title='DP Histogram')
p.quad(top=dp_hist.bin_freq, 
       bottom=0,
       left=dp_hist.bin_edges[:-1], 
       right=dp_hist.bin_edges[1:],
       fill_color="navy",
       line_color="white",
       alpha=0.5,
       legend='DP')

hl.plot.show(p)

## Variant QC

Hail has the function [hl.variant_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc) to compute a list of useful statistics about **variants** from sequencing data.

Once again, **Click the link** above to see the documentation!

In [44]:
mt = hl.variant_qc(mt)

In [45]:
mt.describe(widget=True)

We can `show()` the computed information:

In [46]:
mt.variant_qc.show()

+---------------+------------+--------------------------+
| locus         | alleles    | variant_qc.dp_stats.mean |
+---------------+------------+--------------------------+
| locus<GRCh37> | array<str> |                  float64 |
+---------------+------------+--------------------------+
| 1:904165      | ["G","A"]  |                 7.44e+00 |
| 1:909917      | ["G","A"]  |                 6.13e+00 |
| 1:986963      | ["C","T"]  |                 5.84e+00 |
| 1:1563691     | ["T","G"]  |                 6.77e+00 |
| 1:1707740     | ["T","G"]  |                 8.00e+00 |
| 1:2252970     | ["C","T"]  |                 6.69e+00 |
| 1:2284195     | ["T","C"]  |                 7.69e+00 |
| 1:2779043     | ["T","C"]  |                 5.15e+00 |
| 1:2944527     | ["G","A"]  |                 7.73e+00 |
| 1:3761547     | ["C","A"]  |                 6.76e+00 |
+---------------+------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.dp_stats.stdev | variant_qc.dp_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  3.80e+00 |                1.00e+00 |
|                  3.85e+00 |                1.00e+00 |
|                  3.84e+00 |                1.00e+00 |
|                  4.26e+00 |                1.00e+00 |
|                  4.19e+00 |                1.00e+00 |
|                  4.48e+00 |                1.00e+00 |
|                  4.17e+00 |                1.00e+00 |
|                  2.80e+00 |                1.00e+00 |
|                  4.72e+00 |                1.00e+00 |
|                  3.63e+00 |                1.00e+00 |
+---------------------------+-------------------------+

+-------------------------+--------------------------+
| variant_qc.dp_stats.max | variant_qc.gq_stats.mean |
+-------------------------+--------------------------+
|                 float64 |                  float64 |
+-------------------------+--------------------------+
|                2.10e+01 |                 2.84e+01 |
|                2.30e+01 |                 1.87e+01 |
|                1.90e+01 |                 1.84e+01 |
|                2.00e+01 |                 1.87e+01 |
|                2.60e+01 |                 3.47e+01 |
|                2.40e+01 |                 1.98e+01 |
|                2.20e+01 |                 3.62e+01 |
|                1.50e+01 |                 2.44e+01 |
|                3.10e+01 |                 3.64e+01 |
|                2.40e+01 |                 2.75e+01 |
+-------------------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.gq_stats.stdev | variant_qc.gq_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  2.28e+01 |                3.00e+00 |
|                  1.21e+01 |                3.00e+00 |
|                  1.19e+01 |                2.00e+00 |
|                  1.32e+01 |                0.00e+00 |
|                  2.62e+01 |                3.00e+00 |
|                  1.39e+01 |                2.00e+00 |
|                  2.92e+01 |                0.00e+00 |
|                  2.26e+01 |                0.00e+00 |
|                  3.01e+01 |                3.00e+00 |
|                  1.65e+01 |                3.00e+00 |
+---------------------------+-------------------------+

+-------------------------+---------------+---------------------+
| variant_qc.gq_stats.max | variant_qc.AC | variant_qc.AF       |
+-------------------------+---------------+---------------------+
|                 float64 | array<int32>  | array<float64>      |
+-------------------------+---------------+---------------------+
|                9.90e+01 | [470,60]      | [8.87e-01,1.13e-01] |


Metrics like `call_rate` are important for QC. Let's plot the cumulative density function of call rate per variant:

In [47]:
mt.filter_rows(hl.agg.call_stats(mt.GT, mt.alleles).AF[1] > 0.01).show()

,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"
1:2284195,"[""T"",""C""]"
1:2779043,"[""T"",""C""]"
1:2944527,"[""G"",""A""]"
1:3761547,"[""C"",""A""]"
1:3803755,"[""T"",""C""]"


Let's interrogate the per-variant call rates a bit. We can look at summary statistics:

In [48]:
mt.aggregate_rows(hl.agg.stats(mt.variant_qc.call_rate))

Struct(mean=0.9905656122695046, stdev=0.017914000882037602, min=0.5543071161048689, max=1.0, n=10879, sum=10776.36329587994)

Or we could look at a histogram:

In [49]:
p = hl.plot.histogram(mt.variant_qc.call_rate, title='Call Rate')
hl.plot.show(p)

Before filtering variants, let's check how many variants we started with:

In [50]:
# pre-qc variant count
mt.count_rows()

10879

`filter_rows` removes entire rows of the matrix table. Here, we keep rows where the `call_rate` is over 95%:

In [51]:
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.95)

After filtering, we can see more resolution of the top end of the call rate distribution:

In [52]:
p = hl.plot.histogram(mt.variant_qc.call_rate, title='Call Rate of surviving variants')
hl.plot.show(p)

We can then compute the final sample and variant count:

In [53]:
mt.count()

(10626, 267)

# Association Testing and PCA

We will first filter to common variants (those with an alternate allele frequency over 1%). GWAS cannot detect signal from extremely rare variants, like those only observed in one individual.

In [54]:
mt = mt.filter_rows(hl.agg.call_stats(mt.GT, mt.alleles).AF[1] > 0.01)

In this association study, we'll use fit an independent model for each variant, $v$, of the form:

$$y_s = \beta_v g_{s,v} + \varepsilon ~~~~~~~~~~ \varepsilon \sim N(0, 1)$$

Hail calls this per-row (per-variant) linear model: [hl.linear_regression_rows](https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows).

We use the phenotype `CaffeineConsumption` as our dependent variable, the number of alternate alleles as our independent variable, and no covariates besides an intercept term (that's the `1.0`).

In [55]:
gwas = hl.linear_regression_rows(y=mt.pheno.CaffeineConsumption, 
                                 x=mt.GT.n_alt_alleles(), 
                                 covariates=[1.0])

2023-01-11 14:01:26.152 Hail: INFO: Coerced sorted dataset
2023-01-11 14:01:28.045 Hail: INFO: linear_regression_rows: running on 267 samples for 1 response variable y,
    with input variable x, and 1 additional covariate...
2023-01-11 14:01:31.086 Hail: INFO: wrote table with 8942 rows in 4 partitions to /tmp/persist_tableQcDFTSbhG9
    Total size: 577.29 KiB
    * Rows: 577.28 KiB
    * Globals: 11.00 B
    * Smallest partition: 2163 rows (140.02 KiB)
    * Largest partition:  2297 rows (148.34 KiB)


In [56]:
gwas.describe(widget=True)

## Visualization

Let’s visualize our association test results from the linear regression. We can do so by creating 2 common plots: a [Manhattan plot](https://en.wikipedia.org/wiki/Manhattan_plot) and a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot).

We'll start with the Manhattan plot:

In [58]:
p = hl.plot.manhattan(gwas.p_value)
hl.plot.show(p)

The other common plot is the Q-Q (quantile-quantile) plot.

In [61]:
p = hl.plot.qq(gwas.p_value, height=400, width=400)
hl.plot.show(p)

2023-01-11 14:02:30.344 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-01-11 14:02:31.005 Hail: INFO: wrote table with 8942 rows in 4 partitions to /tmp/persist_tableXbAzMNmm7D
2023-01-11 14:02:32.564 Hail: INFO: wrote table with 8942 rows in 4 partitions to /tmp/persist_table7pGsIk3a6M


## Confounded!

The Q-Q plot indicates **extreme** inflation of p-values.

What do you think is a good range for genomic inflation?

If you've done a GWAS before, you've probably included a few other covariates that might be confounders -- age, sex, and principal components.

Principal components are a measure of genetic ancestry, and can be used to control for [population stratification](https://en.wikipedia.org/wiki/Population_stratification).

Principal component analysis (PCA) is a very general statistical method for reducing high dimensional data to a small number of dimensions which capture most of the variation in the data. Hail has the function [pca](https://hail.is/docs/0.2/methods/stats.html#hail.methods.pca) for performing generic PCA.

PCA is most interpretable after normalization. Hail provides the specialized function [hwe_normalized_pca](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca) which first normalizes the genotypes according to the Hardy-Weinberg Equilibium model.

In [62]:
pca_eigenvalues, pca_scores, _ = hl.hwe_normalized_pca(mt.GT, compute_loadings=False)

2023-01-11 14:02:44.360 Hail: INFO: hwe_normalize: found 8936 variants after filtering out monomorphic sites.
2023-01-11 14:02:46.885 Hail: INFO: pca: running PCA with 10 components...
2023-01-11 14:02:52.199 Hail: INFO: wrote table with 0 rows in 0 partitions to /tmp/persist_tableO3oejD53Xm
    Total size: 22.76 KiB
    * Rows: 0.00 B
    * Globals: 22.76 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A


It's good practice to save the results of expensive computation (like principal components analysis). Hail's Table and Matrix Tables *do not* keep data in memory. They are more like recipes: they describe how to compute a result. Every time you `show`, `collect`, or `write`, Hail will execute the entire recipe from start to finish. We can avoid unnecessary recomputation by writing (saving) our results in a file.

In [63]:
pca_scores.write('resources/pca_scores.ht', overwrite=True)
pca_scores = hl.read_table('resources/pca_scores.ht')

2023-01-11 14:02:52.716 Hail: INFO: Coerced sorted dataset
2023-01-11 14:02:53.698 Hail: INFO: wrote table with 267 rows in 16 partitions to resources/pca_scores.ht


The pca function returns three things:
* **eigenvalues**: an array of doubles
* **scores**: a table keyed by sample
* **loadings**: a table keyed by variant

The **loadings** are the *principal directions*, each of which is a weighted sum of variants (a "virtual variant"). By default, `hwe_normalized_pca` gives us 10 principal directions.

In [64]:
mt = mt.annotate_cols(pca = pca_scores[mt.s])

## Control confounders and run another GWAS

Now that we have computed principal components and saved it into our `mt`, let’s attempt to adjust the inflation that we saw in our initial Q-Q plot e.g.  `mt.pheno.is_female`. We will now add PCs as `covariates` in `linear_regression_rows`:


In [65]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption, 
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pca.scores[0], mt.pca.scores[1], mt.pca.scores[2]])

2023-01-11 14:03:07.421 Hail: INFO: Coerced sorted dataset
2023-01-11 14:03:09.019 Hail: INFO: linear_regression_rows: running on 267 samples for 1 response variable y,
    with input variable x, and 4 additional covariates...
2023-01-11 14:03:10.120 Hail: INFO: wrote table with 8942 rows in 4 partitions to /tmp/persist_tableHayrcKAa8y
    Total size: 575.43 KiB
    * Rows: 575.42 KiB
    * Globals: 11.00 B
    * Smallest partition: 2163 rows (139.58 KiB)
    * Largest partition:  2297 rows (147.85 KiB)


In [66]:
p = hl.plot.qq(gwas.p_value)
hl.plot.show(p)

2023-01-11 14:03:10.457 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-01-11 14:03:11.325 Hail: INFO: wrote table with 8942 rows in 4 partitions to /tmp/persist_tablevqPwJVBFMs
2023-01-11 14:03:12.231 Hail: INFO: wrote table with 8942 rows in 4 partitions to /tmp/persist_tableGUkzgJsNrw


The above Q-Q plot indicates much better (but not good!) genomic control. Let's try the Manhattan plot:

In [67]:
p = hl.plot.manhattan(gwas.p_value)
hl.plot.show(p)

### What other covariates can you think off that could possibly clean up this analysis?

#### Activity

Assemble yourselves into small groups and tackle one of the two following challenges.

**_Purple Hair_**

Create a model with **purple hair** as the outcome


**_Polydactylism_**

Create a model with **six toes** as the outcome

1) What is the distribution of people who have the phenotype?

2) Create a logistic model with the given phenotype outcome using [Hail documentation](https://hail.is/docs/0.2/methods/stats.html#hail.methods.logistic_regression_rows). Use the search function at the top of the documentation page if you need more information!  

3) Generate a QQ plot

4) Generate a Manhattan plot

We'll spend ten minutes on this assignment and then come back together and have a group per-phenotype share their analysis and results.

In [ ]:
mt.describe()